In [1]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
# import pprint as pp
import json
from sqlalchemy import create_engine

In [19]:
  #######DB Setup and Loading#########
#connection_string = "postgres:Password@localhost:5432/zillow"
connection_string = "postgres:apartment@homeslice.cjnrjw08kldx.us-east-2.rds.amazonaws.com:5432/HomeSliceDB"
engine = create_engine(f'postgresql://{connection_string}')

In [5]:
#----API Reference GitHub --> https://github.com/seme0021/python-zillow

api = zillow.ValuationApi()
key = "X1-ZWz17fdl35adqj_3bebs"

In [6]:
#TEST ADDRESS: 14 Hayes Ave, Elizabeth, NJ 07202

address = "501 Shumaker Dr, Monroeville, PA"
postal_code = "15146"

request_data = api.GetSearchResults(key, address, postal_code)

In [7]:
result = request_data.get_dict()
result

{'zpid': '11626766',
 'similarity_score': None,
 'links': {'home_details': 'https://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/',
  'graphs_and_data': 'http://www.zillow.com/homedetails/501-Shumaker-Dr-Monroeville-PA-15146/11626766_zpid/#charts-and-data',
  'map_this_home': 'http://www.zillow.com/homes/11626766_zpid/',
  'comparables': 'http://www.zillow.com/homes/comps/11626766_zpid/'},
 'full_address': {'street': '501 Shumaker Dr',
  'zipcode': '15146',
  'city': 'Monroeville',
  'state': 'PA',
  'latitude': '40.456694',
  'longitude': '-79.763917'},
 'zestimate': {'amount': 199524,
  'amount_currency': 'USD',
  'amount_last_updated': '04/08/2020',
  'amount_change_30days': 7174,
  'valuation_range_low': 183562,
  'valuation_range_high': 223467},
 'local_realestate': {'region_name': 'Monroeville',
  'region_id': '46582',
  'region_type': 'city',
  'overview_link': 'http://www.zillow.com/local-info/PA-Monroeville/r_46582/',
  'fsbo_link': 'http://www

In [8]:
def Merge(dict1, dict2): 
    return(dict2.update(dict1)) 

In [9]:
zestimate = result['zestimate']
zestimate

{'amount': 199524,
 'amount_currency': 'USD',
 'amount_last_updated': '04/08/2020',
 'amount_change_30days': 7174,
 'valuation_range_low': 183562,
 'valuation_range_high': 223467}

In [10]:
address = result['full_address']
address

{'street': '501 Shumaker Dr',
 'zipcode': '15146',
 'city': 'Monroeville',
 'state': 'PA',
 'latitude': '40.456694',
 'longitude': '-79.763917'}

In [11]:
local = result['local_realestate']

In [12]:
merge_data = Merge(zestimate, local)
merge_data

In [13]:
test_log = pd.DataFrame.from_dict(zestimate, orient='index').T
test_log

,amount,amount_currency,amount_last_updated,amount_change_30days,valuation_range_low,valuation_range_high
0,199524,USD,04/08/2020,7174,183562,223467


In [18]:
  #######DB Setup and Loading#########
#connection_string = "postgres:Password@localhost:5432/zillow"
connection_string = "postgres:apartment@homeslice.cjnrjw08kldx.us-east-2.rds.amazonaws.com:5432/HomeSliceDB"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
address = "PA"
add_1 = '"{}"'.format(address)

add_1


'"PA"'

In [31]:
#Function to call data from Zillow and return dataframe


def address_search(address, postal_code):
    request_data = api.GetSearchResults(key, address, postal_code)
    result = request_data.get_dict()
    address = result['full_address']
    
    zestimate = result['zestimate']
    comb_dict = dict(zestimate)
    comb_dict.update(address)
    combined_df = pd.DataFrame.from_dict(comb_dict, orient='index').T

    combined_df['risk_adj_value'] = (combined_df['amount']*.85) #Risk adjusting home by 15%, reducing home val by 15%
    combined_df['max_cash_limit'] = combined_df['risk_adj_value']*.30 #Cannot buy more than 30% of someones home equity, bc then youre pushing majority ownership
    combined_df.to_sql(name='zillow', con=engine, if_exists='append', index=False)
    final=combined_df["amount"]
    test = combined_df["max_cash_limit"]
    
    return(final,test)


In [32]:
address_search("501 Shumaker Dr, Monroeville, PA","15146")


(0    199524
 Name: amount, dtype: object,
 0    50878.6
 Name: max_cash_limit, dtype: object)

In [14]:
#Risk adjustment calculation

master_df['risk_adj_value'] = (master_df['amount']*.85) #Risk adjusting home by 15%, reducing home val by 15%
master_df['max_cash_limit'] = master_df['risk_adj_value']*.30 #Cannot buy more than 30% of someones home equity, bc then youre pushing majority ownership
master_df

#######DB Setup and Loading#########
connection_string = "postgres:Password@localhost:5432/zillow"
#connection_string = "postgres:M@cboy089@localhost:5432/zillow"
engine = create_engine(f'postgresql://{connection_string}')
master_df.to_sql(name='zillow', con=engine, if_exists='replace', index=False)

,amount,amount_currency,amount_last_updated,amount_change_30days,valuation_range_low,valuation_range_high,street,zipcode,city,state,latitude,longitude,risk_adj_value,max_cash_limit
0,493357,USD,04/07/2020,-1195,453888,552560,317 E 8th Ave,08260,Wildwood,NJ,39.00134,-74.794258,419353,125806


In [20]:
connection_string = "postgres:Password@localhost:5432/zillow"

#connection_string = "postgres:M@cboy089@localhost:5432/zillow"
engine = create_engine(f'postgresql://{connection_string}')
master_df.to_sql(name='zillow', con=engine, if_exists='replace', index=False)
